In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List


In [3]:
from src.tool import Function, Tool, ToolKit

In [4]:
def sum2(a: float, b: float) -> float:
    return a + b

def sub(a: float, b: float) -> float:
    return a - b

def list_mul(a: int, b: List = [1,2,3]) -> list:
    return a * b

In [5]:
sum2_func = Function(sum2)
sub_func = Function(sub)

In [6]:
sub_func

Function(name=sub, signature=(a: float, b: float) -> float, doc=None)

In [7]:
sum2_func.run({'a': 2, 'b': 3})

5.0

In [8]:
sub_func.run({'a': 2, 'b': 3})

-1.0

In [9]:
sum2_tool = Tool(func=sum2)

In [10]:
sum2_tool.schema

{'name': 'sum2',
 'doc': None,
 'parameters': {'properties': {'a': {'type': 'float'},
   'b': {'type': 'float'}}}}

In [11]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [12]:
assert sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [13]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [14]:
tk = ToolKit(tools=[sum2, sub, list_mul])

In [15]:
tk.tools

[Tool(name=sum2, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=sub, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=list_mul, signature=(a: int, b: List = [1, 2, 3]) -> list, doc=None)]

In [16]:
assert tk.tools[0].run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [17]:
assert tk.get_tool_by_name('sum2').run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [51]:
import json
import re
from pydantic import BaseModel

class ToolCall(BaseModel):
    name: str
    arguments: dict
    id: int

class ToolCallProcessor:
    def __init__(self, message: str):
        self.message = message
        self.call_json = self.parse_multiple_tool_calls(self.message)
        self.tool_calls = [ToolCall(**call) for call in self.call_json]

    def parse_multiple_tool_calls(self, content: str):
        """
        Parse multiple <tool_call> entries from a string and return a list of corresponding dictionaries.

        Parameters
        ----------
        content : str
            A string containing multiple XML-like <tool_call> entries, where each entry contains JSON data.

        Returns
        -------
        list of dict
            A list of dictionaries parsed from the JSON content inside each <tool_call> tag.
        """
        call_request_list = []

        # Use regex to find matches between <tool_call> tags
        for match in re.finditer(r"<tool_call>\n(.+)?\n</tool_call>", content):
            try:
                group_json = json.loads(match.group(1))
                call_request_list.append(group_json)
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError: {e}")
                print(f"Failed to decode JSON: {match.group(1)}")
                continue

        return call_request_list

# Example usage
toolprocessor = ToolCallProcessor("""
<tool_call>
{"name": "summation", "arguments": {"x": 3, "y": 6}, "id": 2}
</tool_call>
<tool_call>
{"name": "search", "arguments": {"query": "what is an LLM on google.com"}, "id": 3}
</tool_call>
""")


[ToolCall(name='summation', arguments={'x': 3, 'y': 6}, id=2),
 ToolCall(name='search', arguments={'query': 'what is an LLM on google.com'}, id=3)]

In [44]:
toolprocessor.name

AttributeError: 'NoneType' object has no attribute 'name'